# 🔑 StrandWeaver — K-Weaver Training (Colab GPU)

Train **4 XGBoost regressors** that predict optimal k-mer sizes from read characteristics.

| Model | Predicts | Typical range |
|-------|----------|---------------|
| `dbg_model.pkl` | DBG construction k | 15–127 |
| `ul_overlap_model.pkl` | Ultra-long overlap k | 201–1001 |
| `extension_model.pkl` | Contig extension k | 21–127 |
| `polish_model.pkl` | Polishing k | 31–127 |

**Fully self-contained** — synthesises genomes, simulates reads, sweeps k-values,
scores assemblies, picks best k, and trains regressors. No external data needed.

### Setup
1. `Runtime → Change runtime type → T4 GPU` (or A100 if available)
2. Run cells top-to-bottom

### Runtime estimates (T4 GPU / High-RAM)
| Preset | Genomes | K-sweep | Estimated time |
|--------|---------|---------|----------------|
| `quick_test` | 10 | 4 k-values | ~3 min |
| `standard` | 50 | 6 k-values | ~15 min |
| `full` | 100 | 8 k-values | ~30 min |

> **ErrorSmith training** is in a separate notebook: `ErrorSmith_Training_Colab.ipynb`

## 1. Environment Setup

In [ ]:
# ── Verify GPU ──────────────────────────────────────────────────────
import subprocess, sys, os

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv,noheader'],
    capture_output=True, text=True
)
if result.returncode == 0:
    print(f"✓ GPU detected: {result.stdout.strip()}")
    GPU_AVAILABLE = True
else:
    print("⚠ No GPU detected — XGBoost will use CPU (still fast, just slower HP search)")
    GPU_AVAILABLE = False

# Check RAM
import psutil
ram_gb = psutil.virtual_memory().total / 1e9
print(f"✓ RAM: {ram_gb:.1f} GB {'(High-RAM ✓)' if ram_gb > 20 else '(consider High-RAM runtime)'}")
print(f"✓ Python: {sys.version.split()[0]}")

In [ ]:
# ── Install dependencies ────────────────────────────────────────────
!pip install -q xgboost scikit-learn numpy pandas optuna pysam biopython

# Clone StrandWeaver (dev branch) and install — pull latest on re-runs
!git clone -b dev https://github.com/pgrady1322/strandweaver.git /content/strandweaver 2>/dev/null || \
    (cd /content/strandweaver && git pull origin dev)
!cd /content/strandweaver && pip install -e . -q

print("\n✓ Dependencies installed")

In [ ]:
# ── Mount Google Drive (for saving results) ─────────────────────────
from google.colab import drive
drive.mount('/content/drive')

# ── Directory layout ────────────────────────────────────────────────
GDRIVE_DIR     = '/content/drive/MyDrive/Colab Notebooks/strandweaver'
KWEAVER_OUT    = '/content/kweaver_training'
MODELS_OUT     = '/content/trained_models'
GDRIVE_KWEAVER = os.path.join(GDRIVE_DIR, 'kweaver_models')

for d in [GDRIVE_DIR, KWEAVER_OUT, MODELS_OUT]:
    os.makedirs(d, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Working dirs created")
print(f"  K-Weaver output:   {KWEAVER_OUT}")
print(f"  Models output:      {MODELS_OUT}")
print(f"  Drive export:       {GDRIVE_DIR}")

In [ ]:
# ── Verify StrandWeaver imports ──────────────────────────────────────
sys.path.insert(0, '/content/strandweaver')

from strandweaver.preprocessing.kweaver_module import (
    FeatureExtractor, ReadFeatures, KmerPrediction, KWeaverPredictor,
)
from strandweaver.io_utils import SeqRead

import xgboost as xgb
import numpy as np
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger('colab_training')

print(f"✓ StrandWeaver imports OK")
print(f"✓ XGBoost {xgb.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ Pandas {pd.__version__}")

## 2. Configuration
Adjust these parameters before running. The **Standard** preset balances quality and runtime (~15 min on T4).

In [ ]:
# ══════════════════════════════════════════════════════════════════════
#  K-WEAVER CONFIGURATION — edit these before running
# ══════════════════════════════════════════════════════════════════════

# ── Preset selector ─────────────────────────────────────────────────
# Options: 'quick_test', 'standard', 'full'
#
# Timing estimates (Colab free tier, T4 GPU):
#   quick_test :  ~3 min  ( 10 genomes × 3 techs × 4 k =   120 k-scores)
#   standard   : ~15 min  ( 50 genomes × 3 techs × 6 k =   900 k-scores)
#   full       : ~30 min  (100 genomes × 3 techs × 8 k = 2,400 k-scores)
#
# k-sweep uses a fast k-mer spectrum proxy (~0.2 s/k), NOT full
# DBG assembly (~45 s/k).  Even 'full' fits comfortably in one session.
KWEAVER_PRESET = 'full'

PRESETS = {
    'quick_test': {
        'num_genomes': 10,
        'min_genome_size': 500_000,
        'max_genome_size': 2_000_000,
        'technologies': ['hifi', 'ont', 'illumina'],
        'k_values': [21, 31, 51, 77],
    },
    'standard': {
        'num_genomes': 50,
        'min_genome_size': 500_000,
        'max_genome_size': 5_000_000,
        'technologies': ['hifi', 'ont', 'illumina'],
        'k_values': [15, 21, 31, 51, 77, 101],
    },
    'full': {
        'num_genomes': 100,
        'min_genome_size': 500_000,
        'max_genome_size': 5_000_000,
        'technologies': ['hifi', 'ont', 'illumina'],
        'k_values': [15, 21, 31, 41, 51, 63, 77, 101],
    },
}

kw_config = PRESETS[KWEAVER_PRESET]
SEED = 42

print(f"✓ K-Weaver preset: {KWEAVER_PRESET}")
print(f"  Genomes:      {kw_config['num_genomes']}")
print(f"  Genome range: {kw_config['min_genome_size']:,} – {kw_config['max_genome_size']:,} bp")
print(f"  Technologies: {kw_config['technologies']}")
print(f"  K sweep:      {kw_config['k_values']}")
print(f"  Total samples: {kw_config['num_genomes'] * len(kw_config['technologies'])}")

## 3. Generate K-Weaver Training Data
This cell runs the full data generation pipeline. Progress is logged per-genome.

In [ ]:
# ── Generate K-Weaver training CSV ───────────────────────────────────
# This calls the generate_kweaver_training_data.py script inline
# so we can monitor progress in the notebook output.

sys.path.insert(0, '/content/strandweaver/strandweaver/training/scripts/')
from generate_kweaver_training_data import generate_kweaver_training_data
from pathlib import Path

import time
t0 = time.time()

kweaver_csv = generate_kweaver_training_data(
    output_dir=Path(KWEAVER_OUT),
    num_genomes=kw_config['num_genomes'],
    min_genome_size=kw_config['min_genome_size'],
    max_genome_size=kw_config['max_genome_size'],
    technologies=kw_config['technologies'],
    k_values=kw_config['k_values'],
    seed=SEED,
)

elapsed = time.time() - t0
print(f"\n✅ K-Weaver training CSV: {kweaver_csv}")
print(f"   Elapsed: {elapsed/60:.1f} min")

In [ ]:
# ── Inspect the generated data ───────────────────────────────────────
df_kw = pd.read_csv(kweaver_csv)
print(f"Shape: {df_kw.shape}")
print(f"\nTarget distribution:")
for col in ['best_dbg_k', 'best_ul_k', 'best_extension_k', 'best_polish_k']:
    print(f"  {col}: mean={df_kw[col].mean():.1f}, std={df_kw[col].std():.1f}, "
          f"range=[{df_kw[col].min()}, {df_kw[col].max()}]")

print(f"\nTechnology breakdown:")
print(df_kw['technology'].value_counts().to_string())

print(f"\nFeature summary (first 5 rows):")
feature_cols = [
    'mean_read_length', 'median_read_length', 'read_length_n50',
    'mean_base_quality', 'estimated_error_rate', 'gc_content',
    'read_type_encoded', 'num_reads',
]
df_kw[feature_cols].head()

## 4. Train K-Weaver XGBoost Regressors
Trains 4 models with **Optuna Bayesian hyperparameter search** (20 trials each) + 5-fold CV on the best config.

In [ ]:
# ── K-Weaver training with Optuna HP search ─────────────────────────
import optuna
import pickle
import json
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

optuna.logging.set_verbosity(optuna.logging.WARNING)

# 19 ReadFeature columns (input features)
KWEAVER_FEATURES = [
    'mean_read_length', 'median_read_length', 'read_length_n50',
    'min_read_length', 'max_read_length', 'read_length_std',
    'mean_base_quality', 'median_base_quality', 'estimated_error_rate',
    'total_bases', 'num_reads',
    'estimated_genome_size', 'estimated_coverage',
    'gc_content', 'gc_std',
    'read_type_encoded', 'is_paired_end',
    'kmer_spectrum_peak', 'kmer_diversity',
]

# 4 target columns (one model each)
KWEAVER_TARGETS = {
    'dbg_model':       'best_dbg_k',
    'ul_overlap_model': 'best_ul_k',
    'extension_model':  'best_extension_k',
    'polish_model':     'best_polish_k',
}

N_OPTUNA_TRIALS = 20
N_CV_FOLDS = 5

# XGBoost device config
XGB_DEVICE = 'cuda' if GPU_AVAILABLE else 'cpu'
XGB_TREE = 'hist'  # gpu_hist is deprecated in XGBoost 2.x — 'hist' auto-uses GPU


def optuna_objective(trial, X, y):
    """Optuna objective for XGBoost regression HP search."""
    params = {
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'tree_method': XGB_TREE,
        'device': XGB_DEVICE,
        'random_state': SEED,
    }

    model = xgb.XGBRegressor(**params)
    kf = KFold(n_splits=3, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    return scores.mean()


# ── Prepare data ────────────────────────────────────────────────────
df_kw = pd.read_csv(kweaver_csv)

# Fill NaN in optional features (genome_size, coverage, kmer_spectrum, kmer_diversity)
df_kw[KWEAVER_FEATURES] = df_kw[KWEAVER_FEATURES].fillna(0)

X_all = df_kw[KWEAVER_FEATURES].values
print(f"Training data: {X_all.shape[0]} samples × {X_all.shape[1]} features\n")

# ── Train each model ────────────────────────────────────────────────
kweaver_results = {}
kweaver_save_dir = os.path.join(MODELS_OUT, 'kweaver')
os.makedirs(kweaver_save_dir, exist_ok=True)

for model_name, target_col in KWEAVER_TARGETS.items():
    print(f"{'═'*60}")
    print(f"  Training: {model_name} → {target_col}")
    print(f"{'═'*60}")

    y = df_kw[target_col].values

    # ── Optuna HP search ────────────────────────────────────────
    print(f"  Running Optuna ({N_OPTUNA_TRIALS} trials)...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: optuna_objective(trial, X_all, y),
                   n_trials=N_OPTUNA_TRIALS, show_progress_bar=True)

    best_params = study.best_params
    best_params['tree_method'] = XGB_TREE
    best_params['device'] = XGB_DEVICE
    best_params['random_state'] = SEED

    print(f"  Best MSE (neg): {study.best_value:.4f}")
    print(f"  Best params: depth={best_params['max_depth']}, "
          f"lr={best_params['learning_rate']:.4f}, "
          f"n_est={best_params['n_estimators']}")

    # ── 5-fold CV with best params ──────────────────────────────
    print(f"  Running {N_CV_FOLDS}-fold CV with best params...")
    model = xgb.XGBRegressor(**best_params)
    kf = KFold(n_splits=N_CV_FOLDS, shuffle=True, random_state=SEED)

    cv_mse = []
    cv_r2 = []
    cv_mae = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_all)):
        X_train, X_val = X_all[train_idx], X_all[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict(X_val)

        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)

        cv_mse.append(mse)
        cv_r2.append(r2)
        cv_mae.append(mae)
        print(f"    Fold {fold+1}: MSE={mse:.2f}, MAE={mae:.2f}, R²={r2:.4f}")

    print(f"  CV Summary: MSE={np.mean(cv_mse):.2f}±{np.std(cv_mse):.2f}, "
          f"R²={np.mean(cv_r2):.4f}±{np.std(cv_r2):.4f}, "
          f"MAE={np.mean(cv_mae):.2f}")

    # ── Final model (train on all data) ─────────────────────────
    final_model = xgb.XGBRegressor(**best_params)
    final_model.fit(X_all, y)

    # Save
    pkl_path = os.path.join(kweaver_save_dir, f'{model_name}.pkl')
    with open(pkl_path, 'wb') as f:
        pickle.dump(final_model, f)
    print(f"  ✓ Saved: {pkl_path}")

    kweaver_results[model_name] = {
        'target': target_col,
        'best_params': best_params,
        'cv_mse_mean': float(np.mean(cv_mse)),
        'cv_mse_std': float(np.std(cv_mse)),
        'cv_r2_mean': float(np.mean(cv_r2)),
        'cv_r2_std': float(np.std(cv_r2)),
        'cv_mae_mean': float(np.mean(cv_mae)),
        'cv_mae_std': float(np.std(cv_mae)),
        'n_samples': len(y),
    }
    print()

# Save results summary
results_path = os.path.join(kweaver_save_dir, 'training_results.json')
with open(results_path, 'w') as f:
    json.dump(kweaver_results, f, indent=2, default=str)

print(f"\n{'═'*60}")
print(f"  K-Weaver training complete!")
print(f"  Models saved to: {kweaver_save_dir}")
print(f"{'═'*60}")

## 5. Validate K-Weaver Models
Quick sanity check: load the saved models and run inference on a synthetic sample.

In [ ]:
# ── Validate: load models and run sample predictions ─────────────────
print("Loading trained K-Weaver models for validation...\n")

# Load all 4 models
kw_models = {}
for model_name in KWEAVER_TARGETS:
    pkl_path = os.path.join(kweaver_save_dir, f'{model_name}.pkl')
    with open(pkl_path, 'rb') as f:
        kw_models[model_name] = pickle.load(f)
    print(f"  ✓ Loaded {model_name}")

# UL applicability threshold (must match kweaver_module.UL_MIN_N50)
UL_MIN_N50 = 50_000

# Test predictions on 3 representative samples from training data
print(f"\n{'─'*70}")
print(f"{'Technology':<12} {'DBG k':>8} {'UL k':>8} {'Ext k':>8} {'Polish k':>8}  UL status")
print(f"{'─'*70}")

test_samples = df_kw.groupby('technology').first().reset_index()
for _, row in test_samples.iterrows():
    x = row[KWEAVER_FEATURES].values.reshape(1, -1).astype(np.float64)
    preds = {name: int(round(model.predict(x)[0])) for name, model in kw_models.items()}
    tech = row['technology']

    # Determine UL applicability from read_length_n50 feature
    n50 = row.get('read_length_n50', 0)
    if n50 >= UL_MIN_N50:
        ul_status = "✓ applicable"
    else:
        ul_status = f"⚠ advisory (N50={n50:,.0f} bp < 50 Kb)"

    print(f"{tech:<12} {preds['dbg_model']:>8} {preds['ul_overlap_model']:>8} "
          f"{preds['extension_model']:>8} {preds['polish_model']:>8}  {ul_status}")

print(f"{'─'*70}")
print(f"\n✓ K-Weaver validation complete — predictions look reasonable!")
print(f"\n  ℹ UL overlap k is only reliable for ultra-long reads (N50 ≥ 50 Kb).")
print(f"    Reads with shorter N50 should not be fed into the UL overlap slot;"  )
print(f"    doing so is likely to produce mis-joins and fragmented scaffolds.")

## 6. Feature Importance

In [ ]:
# ── Feature importance plots ─────────────────────────────────────────
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Feature Importance — K-Weaver Models', fontsize=16, fontweight='bold')

for idx, (model_name, target_col) in enumerate(KWEAVER_TARGETS.items()):
    ax = axes[idx // 2][idx % 2]
    model = kw_models[model_name]
    importances = model.feature_importances_
    sorted_idx = np.argsort(importances)[-10:]  # Top 10
    ax.barh(range(10), importances[sorted_idx], color='steelblue')
    ax.set_yticks(range(10))
    ax.set_yticklabels([KWEAVER_FEATURES[i] for i in sorted_idx], fontsize=9)
    ax.set_title(f'{model_name}', fontsize=11)
    ax.set_xlabel('Importance')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(os.path.join(MODELS_OUT, 'kweaver_feature_importance.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✓ Feature importance plot saved")

## 7. Export Models to Google Drive

In [ ]:
# ── Package and export to Google Drive ────────────────────────────────
import tarfile, shutil, glob

GDRIVE_TARBALL = os.path.join(GDRIVE_DIR, 'kweaver_models.tar.gz')

# List all model files
model_files = glob.glob(f'{kweaver_save_dir}/**/*', recursive=True)
print(f"Model files to export ({len(model_files)}):")
for f in sorted(model_files):
    if os.path.isfile(f):
        size_kb = os.path.getsize(f) / 1024
        print(f"  {os.path.relpath(f, kweaver_save_dir):40s} {size_kb:>8.1f} KB")

# Create tarball
print(f"\nCreating tarball...")
with tarfile.open(GDRIVE_TARBALL, 'w:gz') as tar:
    tar.add(kweaver_save_dir, arcname='kweaver_models')

tarball_size = os.path.getsize(GDRIVE_TARBALL) / (1024 * 1024)
print(f"\n✅ Exported to Google Drive:")
print(f"   {GDRIVE_TARBALL}")
print(f"   Size: {tarball_size:.1f} MB")

# Also copy to Drive for easy access
if os.path.exists(GDRIVE_KWEAVER):
    shutil.rmtree(GDRIVE_KWEAVER)
shutil.copytree(kweaver_save_dir, GDRIVE_KWEAVER)
print(f"   Copied → {GDRIVE_KWEAVER}")

print(f"\n📥 To download locally:")
print(f"   1. Open Google Drive → My Drive/Colab Notebooks/strandweaver/")
print(f"   2. Download kweaver_models.tar.gz")
print(f"   3. Extract to strandweaver/trained_models/kweaver/")

## 8. Summary

In [ ]:
# ── Final summary ────────────────────────────────────────────────────
print("╔══════════════════════════════════════════════════════════════╗")
print("║       K-Weaver Model Training — Summary                     ║")
print("╠══════════════════════════════════════════════════════════════╣")
print("║                                                              ║")
print("║  4 XGBoost Regressors:                                       ║")
print("║  ─────────────────────                                       ║")
for name, res in kweaver_results.items():
    print(f"║    {name:20s}  R²={res['cv_r2_mean']:.3f}±{res['cv_r2_std']:.3f}  "
          f"MAE={res['cv_mae_mean']:.1f}  ║")
print("║                                                              ║")
print("║  Output files on Google Drive:                               ║")
print(f"║    {GDRIVE_DIR}/")
print(f"║    ├── kweaver_models/       (4 .pkl + results JSON)")
print(f"║    └── kweaver_models.tar.gz")
print("║                                                              ║")
print("║  Next steps:                                                 ║")
print("║    1. Download models to local strandweaver/trained_models/  ║")
print("║    2. Run ErrorSmith_Training_Colab.ipynb for error models   ║")
print("║    3. Then: strandweaver core-assemble --use-ai <reads>      ║")
print("║                                                              ║")
print("╚══════════════════════════════════════════════════════════════╝")